---

## Transformer Model v1

---

Basic usage of a Transformer.

In [ ]:
debug = True
DRIVE_PATH = "/content/drive/MyDrive/data606"

# Set the location of this script in GDrive
SCRIPT_PATH = DRIVE_PATH + "/src/"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd $SCRIPT_PATH

/content/drive/MyDrive/data606/src


In [ ]:
# Load model classes
%run -i "./Model_Base.ipynb"

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense,RepeatVector, LSTM, Dropout, Reshape
from tensorflow.keras.layers import Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.layers import GlobalAveragePooling1D,LayerNormalization,MultiHeadAttention
from keras.callbacks import Callback, EarlyStopping
from tensorflow.keras import Input,Model

In [ ]:

class Model_Transformerv1(Base_Model):
  """
  Constructs a model instance. Input data should be a dataframe containing
  only the data to be modeled.
  """
  EPSILON = 1e-9

  def get_name(self):
    return "TXERv1"

  def train(self, X_train=None, y_train=None, num_features=None, dataset=None):
    """
    Declare model and train.
    Based on https://keras.io/examples/timeseries/timeseries_classification_from_scratch/.
    """
    if (dataset is not None):
      print(f'### Fitting TXERv1 model to ds {dataset}')
      # get batch size
      for X,y in dataset:
        batch_size = X.shape[-2]
        assert not np.any(np.isnan(X))
        assert not np.any(np.isnan(y))
    else:
      print(f'### Fitting TXERv1 model to X {X_train.shape}, y {y_train.shape}')

    head_size=256
    num_heads=4  # was 4
    ff_dim=4 #batch_size  # was 4
    num_transformer_blocks=4
    mlp_units=[128]
    mlp_dropout=0.2
    dropout=0.25

    print(f'### num features: {num_features}')
    print(f'### batch size: {batch_size}')

    early_stop = EarlyStopping(monitor = "loss", mode = "min", patience = 25)

    inputs = Input(shape=(batch_size,num_features))
    x = inputs
    print(f'### input X: {x}')
    # Build txer blocks
    for _ in range(num_transformer_blocks):
        x = self.transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
    # print(f'### after txfrs: {x.shape}')

    x = GlobalAveragePooling1D(data_format="channels_first")(x)
    # print(f'### after GlobalVgPool: {x.shape}')
    for dim in mlp_units:
        x = Dense(dim, activation="relu")(x)
        x = Dropout(mlp_dropout)(x)
    # print(f'### after mlp: {x.shape}')
    # following used softmax; removed during troubleshooting, and probly ok as is
    outputs = Dense(self.NUM_LABELS*self.LABEL_WINDOW, activation="relu")(x)
    # print(f'### after dense: {outputs.shape}')

    if (self.LABEL_WINDOW > 1):
     # reshape as => [batch, out_steps, labels]
     # print(f'### Reshaping output: {outputs.shape}')
     outputs = Reshape([self.LABEL_WINDOW, self.NUM_LABELS])(outputs)
     print(f'### Reshaped output: {outputs.shape}')

    model = Model(inputs, outputs)
    # print(f'## Compiling model')
    model.compile(
        #loss="sparse_categorical_crossentropy",
        loss= tf.keras.losses.Huber(),
        optimizer=tf.keras.optimizers.Adam(learning_rate=self.ALPHA),  # orig 1e-4
        metrics=["sparse_categorical_accuracy"],
        run_eagerly=False,  # for debugging
    )
    model.summary()

    if (dataset is not None):
      self.model_hist = model.fit(dataset, epochs=self.NUM_EPOCHS, callbacks = [early_stop], verbose=1)
    else:
      self.model_hist = model.fit(X_train, y_train, epochs=self.NUM_EPOCHS, verbose=1, callbacks = [early_stop] )

    self.model = model
    return self.model_hist

  def transformer_encoder(self, inputs, head_size, num_heads, ff_dim, dropout=0):
    """
    A transformer block. Usually there are about 4 of these in a complete model.
    """
    # Normalization and Attention
    # print(f'### transformer inputs: {inputs.shape}')
    x = LayerNormalization(epsilon=self.EPSILON)(inputs)
    # print(f'### after Norm x: {x.shape}')
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    # print(f'### after MultiHead x: {x.shape}')
    x = Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = LayerNormalization(epsilon=self.EPSILON)(res)
    # print(f'### after Norm 2: {x.shape}')
    # x = Conv1D(filters=inputs.shape[-1], kernel_size=1, activation="relu")(x)
    x = Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    # print(f'### after conv: {x.shape}')
    x = Dropout(dropout)(x)
    x = Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    # print(f'### after conv 2: {x.shape}')
    return x + res